# Práctica: Los Juegos del Hambre

ESTUDIANTE: LORENZO OTERO RAMOS

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

En esta práctica vamos a enfrentarnos a un problema desafiante de clasificación de imágenes, construyendo una red neuronal profunda que sea capaz de clasificar entre diferentes tipos de comida. ¡Que comiencen los Juegos del Hambre!

## Guidelines

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema, y a conseguir una calificación más alta. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb), y de haber [deactivado otras sesiones que tuvieras activas](https://stackoverflow.com/a/53441194/2436578).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
!pwd 


/content


In [5]:
import numpy as np
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Obtención de datos

Vamos a usar un dataset de imágenes de comida disponible en [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). Para descargarlo, necesitarás crear una cuenta de usuario en Kaggle, y obtener tus credenciales de la API. Puedes hacerlo siguiendo las instrucciones de [esta sección](https://github.com/Kaggle/kaggle-api#api-credentials). ¡Ojo! Tus credenciales de la API no son lo mismo que la contraseña que utilizas para acceder a tu cuenta en Kaggle.

Una vez tengas el fichero JSON con tus credenciales, puedes declararlas en este notebook asignando las variables de entorno adecuadas, de la siguiente manera

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Cuando lo hayas hecho, podrás descargar el dataset a la máquina donde esté corriendo este notebook usando el siguiente comando

    !kaggle datasets download trolukovich/food11-image-dataset --unzip -p YOUR_LOCAL_FOLDER
    
donde debes indicar el nombre de un directorio válido como "YOUR_LOCAL_FOLDER". Si prefieres descargar los datos en la misma carpeta que este notebook, puedes quitar la parte `-p YOUR_LOCAL_FOLDER` del comando.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Crea tu propia cuenta de Kaggle (si no tienes ya una), obtén tus credenciales, y usa la celda inferior para declarar tu nombre de usuario de Kaggle y tu clave como variables de entorno. A continuación, usa la misma celda para descargar el dataset de imágenes.
    
¡Ojo! Debes mantener estas credenciales en secreto, ya que son personales a tu usuario de Kaggle. Recuerda borrarlas de la celda antes de entregar este notebook.
</font>

***

In [6]:
import os

os.environ["KAGGLE_USERNAME"] = "lorenzooteroramos"
os.environ["KAGGLE_KEY"] = ""

!kaggle datasets download trolukovich/food11-image-dataset --unzip -p data

100% 1.08G/1.08G [00:36<00:00, 34.9MB/s]
100% 1.08G/1.08G [00:36<00:00, 32.1MB/s]


Revisa ahora la carpeta en la que has descargado los datos. Verás que contiene 3 subdirectorios:

* **training**, contiene las imágenes a utilizar para entrenar el modelo.
* **validation**, contiene imágenes adicionales que podrías usar como datos de entrenamiento adicionales, o para algún tipo de estrategia de validación como Early Stopping.
* **evaluation**, contiene las imágenes que debes utilizar para testear el modelo. Las imágenes de esta carpeta **solo** pueden utilizarse para medir el rendimiento del modelo tras su entrenamiento, y para nada más.

Además de esto, dentro de cada una de estas carpetas encontrarás una subcarpeta para cada una de las 11 clases de comida:

* Bread (panes)
* Dairy product (lácteos)
* Dessert (postres)
* Egg (huevos)
* Fried food (fritos)
* Meat (carnes)
* Noodles-Pasta (pasta)
* Rice (arroz)
* Seafood (pescado y marisco)
* Soup (sopas)
* Vegetable-Fruit (vegetales y frutas)

Esta es una forma estándar de organizar los datasets de imágenes: una carpeta para cada clase. Para facilitar los pasos de procesamiento que vendrán a continuación, vamos a definir algunas variables que nos indiquen dónde están almacenados los diferentes conjuntos de datos.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea variables <b>TRAINDIR</b>, <b>VALDIR</b> y <b>TESTDIR</b>, cada una conteniendo una cadena de texto con la ruta al directorio donde están los datos de entrenamiento, validación y evaluación, respectivamente.
</font>

***

In [7]:
TRAINDIR = "/content/data/training"
VALDIR = "/content/data/validation"
TESTDIR = "/content/data/evaluation"

### Reducción de clases

Con el fin de hacer este problema más accesible de cara a la práctica, vamos a centrarnos solo en seis de las clases de comida disponibles: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` y `Meat`. Para ello, se provee el código siguiente, que elimina de los datos descargados las carpetas correspondientes a imágenes de las otras clases.

In [8]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

Deleting /content/data/training/Soup...
Deleting /content/data/training/Noodles-Pasta...
Deleting /content/data/training/Seafood...
Deleting /content/data/training/Vegetable-Fruit...
Deleting /content/data/training/Rice...
Deleting /content/data/evaluation/Soup...
Deleting /content/data/evaluation/Noodles-Pasta...
Deleting /content/data/evaluation/Seafood...
Deleting /content/data/evaluation/Vegetable-Fruit...
Deleting /content/data/evaluation/Rice...
Deleting /content/data/validation/Soup...
Deleting /content/data/validation/Noodles-Pasta...
Deleting /content/data/validation/Seafood...
Deleting /content/data/validation/Vegetable-Fruit...
Deleting /content/data/validation/Rice...


## Procesando imágenes desde ficheros

Este dataset de imágenes es grande, con imágenes de mayor resolución que las que hemos utilizado en el tutorial del MNIST, y cada una de ellas teniendo diferentes tamaños y relación de aspecto. Además, mientras que para el MNIST teníamos una función de keras que preparaba los datos para nosotros, en esta ocasión tendremos que realizar el trabajo de carga y procesamiento de las imágenes.

Una forma conveniente de hacer todo este trabajo es a través de la función Keras `image_dataset_from_directory`. Esta función crea un objeto `Dataset` de TensorFlow con todas las imágenes de un directorio, cargándolas en memoria de forma dinámica solo cuando la red neuronal necesita utilizarlas. Esta función también nos permite especificar algunas opciones de preprocesamiento muy útiles.

Por ejemplo, podemos crear un `Dataset` con los datos en la carpeta de training:

In [9]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

image_size = 32
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Found 6082 files belonging to 6 classes.


Observa los parámetros que se han utilizado para configurar el dataset:

* El **directorio** desde el que cargar las imágenes.
* Un **tamaño de imagen (image_size)** que se utilizará para redimensionar todas las imágenes cargadas a ese tamaño común, en este caso 32x32 píxeles.
* El **tamaños de los lotes (batch_size)** de imágenes a ser generados. Nótese que definimos aquí este parametro en lugar de en el paso `fit` de la red, como hemos hecho en otros ejercicios, porque el objeto `Dataset` resultante hará uso de esta información para mantener en memoria solo algunos batches de imágenes, ahorrando así memoria.
* El **modo de etiquetado (label_mode)**, esto es, la codificación de las etiquetas a utilizar. `categorical` significa que utilizaremos la ya conocida codificación one-hot.

Un objeto `Dataset` funciona de manera muy similar a un generador de Python, lo que significa que podemos iterar sobre él para obtener batches de imágenes ya preprocesadas. Por ejemplo, el siguiente código inicia un bucle para extraer todos los batches del `Dataset`, nos muestra el contenido el primero, y detiene la iteración.

In [10]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

Shape of input batch: (64, 32, 32, 3)
Shape of output batch: (64, 6)
Input batch:
[[[[1.27289062e+02 1.22289062e+02 1.18789062e+02]
   [1.33929688e+02 1.31429688e+02 1.29929688e+02]
   [1.39929688e+02 1.34929688e+02 1.31429688e+02]
   ...
   [1.02929688e+02 7.00703125e+01 5.10000000e+01]
   [1.20710938e+02 9.27109375e+01 7.87109375e+01]
   [1.42359375e+02 1.19359375e+02 1.05359375e+02]]

  [[1.33132812e+02 1.29132812e+02 1.26132812e+02]
   [1.38289062e+02 1.33789062e+02 1.30789062e+02]
   [1.42789062e+02 1.37789062e+02 1.34789062e+02]
   ...
   [1.06398438e+02 7.10000000e+01 5.01328125e+01]
   [9.13671875e+01 6.04765625e+01 4.43906250e+01]
   [1.22867188e+02 9.91328125e+01 8.72656250e+01]]

  [[1.37648438e+02 1.32648438e+02 1.29648438e+02]
   [1.41648438e+02 1.37148438e+02 1.34148438e+02]
   [1.45703125e+02 1.37703125e+02 1.34703125e+02]
   ...
   [8.85546875e+01 6.45000000e+01 4.74453125e+01]
   [4.89062500e+01 3.12031250e+01 2.15000000e+01]
   [8.58906250e+01 6.13515625e+01 4.8203125

Podemos ver que, efectivamente, el generador produce un tensor de datos de entrada de las dimensiones apropiadas para poder introducirlo en la red neuronal, y que las salidas también se han codificado correctamente como one-hot.
No obstante, todavía hay un problema con los datos: los valores de los píxeles están en el rango [0, 255], lo cual puede producir problemas de entrenamiento. Resolveremos este punto después, en la definición de la red neuronal, mediante una capa especial. Por ahora vamos a cotinuar, definiendo una función que construya los `Dataset` para los datos de entrenamiento, validación y test.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea una función <b>create_datasets</b> que reciba los siguiente parámetros:
    <ul>
      <li><b>traindir</b>: el directorio donde están localizadas las imágenes de entrenamiento.</li>
      <li><b>valdir</b>: el directorio donde están localizadas las imágenes de validación.</li>
      <li><b>testdir</b>: el directorio donde están localizadas las imágenes de test.</li>
      <li><b>image_size</b>: el tamaño que se utilizará para redimensionar todas las imágenes a una resolución común.</li>
      <li><b>batch_size</b>: el tamaño de los batches de imágenes que serán generados.</li>
    </ul>
    La función debe crear objetos `Dataset` para los directorios de entrenamiento, validación y test, y devolver los tres datasets creados como

    return train_dataset, val_dataset, test_dataset
</font>

***

In [11]:
def create_datasets(traindir, valdir, testdir, image_size, batch_size):
  
  train_dataset = image_dataset_from_directory(
      traindir,
      image_size = (image_size, image_size),
      batch_size = batch_size,
      label_mode = 'categorical'
  )
  val_dataset = image_dataset_from_directory(
      valdir,
      image_size = (image_size, image_size),
      batch_size = batch_size,
      label_mode = 'categorical'
  )
  test_dataset = image_dataset_from_directory(
      testdir,
      image_size = (image_size, image_size),
      batch_size = batch_size,
      label_mode = 'categorical'
  )
  return train_dataset, val_dataset, test_dataset

Probemos que la función que has implementado funciona correctamente

In [12]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


¡Ahora que tenemos nuestros `Dataset` podemos entrenar una red profunda con ellos! Como ejemplo, vamos a construir una red convolucional extremadamente simple. Nótese cómo hemos añadido una capa especial de preprocesado llamada `Rescaling`, que será la encargada de normalizar los valores de los píxeles al rango [0, 1] cada vez que la red reciba una imagen.

¡Ojo! Esta red tan simple no producirá errores al ejecutar, pero tiene algunos fallos de diseño que deberás corregir cuando crees tu propia red, más adelante en esta práctica.

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Convolution2D, Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

El método `fit` de un modelo Keras puede recibir un objeto `Dataset` como datos de entrenamiento, en lugar de un par de tensores (entradas, salidas). Como al construir los `Dataset` ya especificamos el tamaño de batch, no es necesario indicarlo ahora.

In [14]:
model.fit(train_dataset, epochs=1)

96/96 [==============================] - 17s 75ms/step - loss: 1.7267 - accuracy: 0.2701


Análogamente, podemos evaluar el rendimiento de nuestro modelo sobre el `Dataset` de test de la siguiente manera

In [15]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 111ms/step - loss: 2.3198 - accuracy: 0.2647
Loss 2.32, accuracy 26.5%


Este nivel de acierto puede parecer pobre, pero ten en cuenta que hemos usado un modelo muy simple y que el problema es de 6 clases. ¿Serás capaz de hacerlo mejor?

## Construyendo tu propia red

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Diseña una red neuronal profunda que obtenga el mejor acierto posible sobre los datos de test. Puedes usar los datos de entrenamiento y validación como te parezca mejor, pero <b>sólo</b> puedes usar los datos de test para evaluar el acierto de tu modelo. Debes obtener una red capaz de alcanzar al menos un 45% de acierto sobre los datos de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunas recomendaciones y estrategias que pueden ayudar a mejorar tu diseño de red:
    
- <b>Arquitectura</b>: usa todos los trucos que has aprendido en los ejercicios anteriores: capas convolucionales + pooling, activaciones ReLU, dropout... asegúrate también de utilizar un buen optimizador, con una función de error (loss) adecuada, así como una función de activación en la capa de salida que sea adecuada para esta clase de problema (clasificación multiclase).
- <b>Desarrollo incremental</b>: empieza por redes pequeñas, con un número pequeño de parámetros, de forma que puedas comprobar rápidamente qué tal funcionan. Después, puedes hacer tu red más grande en tres direcciones: mayor tamaño de imágenes de entrada, más capas, y más kernels por capa convolucional o unidades por capa densa. Si aumentas el tamaño de las imágenes de entrada, asegúrate de añadir también más capas Convolution+Pooling, para que así a la capa Flatten solo lleguen imágenes muy pequeñas (10x10 píxeles o menos).
- <b>Tamaño de imágenes</b>: configurar los `Dataset` para que carguen imágenes de mayor tamaño puede mejorar significativamente el rendimiento de tu red. Pero ten cuidado, ¡también puedes encontrarte errores de falta de memoria (CUDA memory error) si cargas imágenes a un tamaño demasiado grande! Para esta práctica, un tamaño mayor a 256 puede ser demasiado grande...
- <b>Controlar el número de épocas</b>: Usa una <a href="https://keras.io/api/callbacks/early_stopping/">**estrategia de EarlyStopping**</a> para monitorizar el loss de los datos de validación, y así detener el entrenamiento cuando tras un número de épocas esa loss no haya decrecido. Configurar la EarlyStopping para restaurar los mejores parámetros encontrados durante la optimización también puede resultarte útil.
- <b>Sobreajuste</b>: si tu red obtiene un accuracy casi perfecto en entrenamiento, puede que estés sufriendo sobreajuste (aunque puede que no...). Prueba a incrementar el nivel de Dropout en las capas Dense para comprobar si así obtienes mejoras en el test.
- <b>Demasiado bueno para ser verdad</b>: si tu red obtiene resultados muy buenos, del orden del 90% o más de acierto en test... sospecha. Es posible que estés mezclando los datos de entrenamiento y test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Como ejercicio avanzado, añade las siguiente estrategias a tu red:

- Usa **técnicas de "image augmentation"** para aumentar artificialmente tu dataset de entrenamiento. Para ello, explora las <a href="https://keras.io/api/layers/preprocessing_layers/image_augmentation/">capas de augmentation disponibles en Keras</a>.
- Usa capas de <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> para facilitar el entrenamiento de la red. Revisa en las diapositivas de clase cuál es la forma adecuada de colocarlas en la red.
    
Usando estos trucos y los mencionados en el punto anterior, es posible obtener más de un 60% de acierto en el conjunto de test.

</font>

***

In [16]:
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import pandas as pd

In [26]:
Nombre = "Red Sencilla"
Diseño = "C(4) + Flatten + Dense(6)"
Estrategia_entrenamiento = "Entrenamiento desde 0"

image_size = 32 #tamaño imagen de entrada
channels = 3 #canales de color
batch_size = 64 #tamaño del batch
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size) #llama a la funcion create_datasets

mlp = Sequential()

mlp.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3))) #capa de rescaling para normalizar valores
mlp.add(Convolution2D (4, 3, activation = 'relu')) #Capa de convulcion 2D de 4 filtros con funcion de activacion RELU
mlp.add(Flatten()) #Capa flatten para aplanar la entrada
mlp.add(Dense(6, activation= 'softmax')) #Capa densa para 6 salidas y softmax

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [27]:
mlp.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 4)         112       
                                                                 
 flatten_3 (Flatten)         (None, 3600)              0         
                                                                 
 dense_3 (Dense)             (None, 6)                 21606     
                                                                 
Total params: 21,718
Trainable params: 21,718
Non-trainable params: 0
_________________________________________________________________


In [28]:
mlp.compile (loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [29]:
mlp.fit(train_dataset, batch_size=64, epochs=5)

Epoch 1/5
96/96 [==============================] - 10s 77ms/step - loss: 1.6642 - accuracy: 0.2841
Epoch 2/5
96/96 [==============================] - 9s 77ms/step - loss: 1.4698 - accuracy: 0.3982
Epoch 3/5
96/96 [==============================] - 9s 89ms/step - loss: 1.3998 - accuracy: 0.4395
Epoch 4/5
96/96 [==============================] - 9s 90ms/step - loss: 1.3615 - accuracy: 0.4530
Epoch 5/5
96/96 [==============================] - 8s 76ms/step - loss: 1.3235 - accuracy: 0.4740


In [30]:
score_mlp = mlp.evaluate(test_dataset)

print("Test loss", score_mlp[0]) #test loss
print("Test accuracy", score_mlp[1]) #Acuracy 0.394. Vamos a aumentar a 10 epochs para que tenga más oportunidades de aprender

33/33 [==============================] - 3s 70ms/step - loss: 1.5058 - accuracy: 0.3947
Test loss 1.5058262348175049
Test accuracy 0.3946859836578369


In [31]:
Nombre = "Red Sencilla (10 epochs)"

mlp.fit(train_dataset, batch_size=64, epochs=10)

score_mlp_10epochs = mlp.evaluate(test_dataset) #Evaluamos en test

print("Test loss", score_mlp_10epochs[0])
print("Test accuracy", score_mlp_10epochs[1]) #El acuracy es practicamente igual

Epoch 1/10
96/96 [==============================] - 10s 95ms/step - loss: 1.2983 - accuracy: 0.4893
Epoch 2/10
96/96 [==============================] - 9s 89ms/step - loss: 1.2677 - accuracy: 0.5041
Epoch 3/10
96/96 [==============================] - 8s 79ms/step - loss: 1.2404 - accuracy: 0.5164
Epoch 4/10
96/96 [==============================] - 9s 83ms/step - loss: 1.2251 - accuracy: 0.5253
Epoch 5/10
96/96 [==============================] - 9s 91ms/step - loss: 1.1973 - accuracy: 0.5388
Epoch 6/10
96/96 [==============================] - 9s 88ms/step - loss: 1.2099 - accuracy: 0.5358
Epoch 7/10
96/96 [==============================] - 8s 74ms/step - loss: 1.1555 - accuracy: 0.5600
Epoch 8/10
96/96 [==============================] - 8s 74ms/step - loss: 1.1331 - accuracy: 0.5750
Epoch 9/10
96/96 [==============================] - 9s 88ms/step - loss: 1.1333 - accuracy: 0.5763
Epoch 10/10
33/33 [==============================] - 3s 70ms/step - loss: 1.5880 - accuracy: 0.3986
Test los

In [32]:
Nombre = "Red_1"
Diseño = "C(32) + Pool(2) + Flatten + Dense(6)"
Estrategia_entrenamiento = "Entrenamiento desde 0"

image_size = 32
channels = 3
batch_size = 64
kernel_size = 3
pool_size = 2
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size)

convol = Sequential()

convol.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3))) #Normalizar imagenes de entrada
convol.add(Convolution2D( #Convulcion2D
    32, 
    (kernel_size, kernel_size),
    padding='valid', 
    input_shape=(image_size, image_size, 1),
    activation="relu"
)) 
convol.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
convol.add(Flatten()) #Aplana en 1D
convol.add(Dense(6, activation="softmax")) #Capa densa de 6 salidas

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [33]:
convol.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 flatten_4 (Flatten)         (None, 7200)              0         
                                                                 
 dense_4 (Dense)             (None, 6)                 43206     
                                                                 
Total params: 44,102
Trainable params: 44,102
Non-trainable params: 0
__________________________________________________

In [34]:
convol.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
convol.fit(
    train_dataset,
    batch_size=64, 
    epochs=20, 
    verbose=2 
)

Epoch 1/20
96/96 - 9s - loss: 1.5901 - accuracy: 0.3428 - 9s/epoch - 91ms/step
Epoch 2/20
96/96 - 8s - loss: 1.4033 - accuracy: 0.4372 - 8s/epoch - 85ms/step
Epoch 3/20
96/96 - 9s - loss: 1.3394 - accuracy: 0.4719 - 9s/epoch - 92ms/step
Epoch 4/20
96/96 - 8s - loss: 1.2863 - accuracy: 0.4928 - 8s/epoch - 88ms/step
Epoch 5/20
96/96 - 8s - loss: 1.2463 - accuracy: 0.5181 - 8s/epoch - 78ms/step
Epoch 6/20
96/96 - 9s - loss: 1.2078 - accuracy: 0.5312 - 9s/epoch - 92ms/step
Epoch 7/20
96/96 - 10s - loss: 1.1735 - accuracy: 0.5567 - 10s/epoch - 100ms/step
Epoch 8/20
96/96 - 9s - loss: 1.1495 - accuracy: 0.5668 - 9s/epoch - 93ms/step
Epoch 9/20
96/96 - 8s - loss: 1.0969 - accuracy: 0.5891 - 8s/epoch - 82ms/step
Epoch 10/20
96/96 - 8s - loss: 1.0593 - accuracy: 0.6056 - 8s/epoch - 82ms/step
Epoch 11/20
96/96 - 9s - loss: 1.0296 - accuracy: 0.6207 - 9s/epoch - 94ms/step
Epoch 12/20
96/96 - 8s - loss: 0.9934 - accuracy: 0.6363 - 8s/epoch - 88ms/step
Epoch 13/20
96/96 - 9s - loss: 0.9647 - accura

In [35]:
score_convol = convol.evaluate(test_dataset)
print("Test loss", score_mlp[0])
print("Test accuracy", score_mlp[1]) #Acuracy del 0.39, implementaremos Dropout para evitar overfitting y detener el modelo.

33/33 [==============================] - 3s 68ms/step - loss: 1.5785 - accuracy: 0.4295
Test loss 1.5058262348175049
Test accuracy 0.3946859836578369


In [36]:
Nombre = "Red_2"
Diseño = "C(32) + C(32) + Pool(2) + Flatten + Dense(128) + Dropout(0.5) + Dense(6)"
Estrategia_entrenamiento = "Entrenamiento desde 0 y Dropout"

image_size = 32
batch_size = 64
kernel_size = 3
pool_size = 2
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size)

convol_dropout = Sequential()

convol_dropout.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3))) #Normalizando pixeles
convol_dropout.add(Convolution2D (32,  #Capa de convulcion 2D
                              (kernel_size, kernel_size),
                              padding='valid',
                              input_shape=(image_size, image_size, 3), 
                              activation='relu'
                              ))

convol_dropout.add(Convolution2D (32, (kernel_size, kernel_size), activation='relu'))
convol_dropout.add(MaxPooling2D(pool_size=(pool_size, pool_size))) #reduccion de dimensionalidad
convol_dropout.add(Flatten()) #Aplanar las salidas a vector 1D
convol_dropout.add(Dense(128, activation="relu")) #Capa dense de 128 neuronas con activacion relu
convol_dropout.add(Dropout(0.5)) #Dropout para prevenir el sobreajuste
convol_dropout.add(Dense(6, activation="softmax")) #Capa densa de salida de 6 neuronas

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [37]:
convol_dropout.summary() 

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 6272)              0         
                                                                 
 dense_5 (Dense)             (None, 128)               802944    
                                                      

In [38]:
convol_dropout.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [39]:
convol_dropout.fit(
    train_dataset, 
    batch_size=64, 
    epochs=20, 
    verbose=2 )

Epoch 1/20
96/96 - 10s - loss: 1.6589 - accuracy: 0.3035 - 10s/epoch - 108ms/step
Epoch 2/20
96/96 - 9s - loss: 1.5170 - accuracy: 0.3594 - 9s/epoch - 91ms/step
Epoch 3/20
96/96 - 8s - loss: 1.4346 - accuracy: 0.4153 - 8s/epoch - 87ms/step
Epoch 4/20
96/96 - 8s - loss: 1.3683 - accuracy: 0.4410 - 8s/epoch - 78ms/step
Epoch 5/20
96/96 - 7s - loss: 1.3068 - accuracy: 0.4778 - 7s/epoch - 77ms/step
Epoch 6/20
96/96 - 9s - loss: 1.2512 - accuracy: 0.4964 - 9s/epoch - 89ms/step
Epoch 7/20
96/96 - 9s - loss: 1.1906 - accuracy: 0.5265 - 9s/epoch - 89ms/step
Epoch 8/20
96/96 - 7s - loss: 1.1380 - accuracy: 0.5515 - 7s/epoch - 77ms/step
Epoch 9/20
96/96 - 9s - loss: 1.0743 - accuracy: 0.5763 - 9s/epoch - 89ms/step
Epoch 10/20
96/96 - 9s - loss: 0.9863 - accuracy: 0.6118 - 9s/epoch - 89ms/step
Epoch 11/20
96/96 - 9s - loss: 0.9116 - accuracy: 0.6465 - 9s/epoch - 91ms/step
Epoch 12/20
96/96 - 7s - loss: 0.8651 - accuracy: 0.6620 - 7s/epoch - 76ms/step
Epoch 13/20
96/96 - 9s - loss: 0.7707 - accura

In [40]:
score_convol_dropout = convol_dropout.evaluate(test_dataset)
print("Test loss", score_convol_dropout[0])
print("Test accuracy", score_convol_dropout[1]) #acuray de 0.46. 

33/33 [==============================] - 3s 69ms/step - loss: 1.9923 - accuracy: 0.4671
Test loss 1.9922899007797241
Test accuracy 0.4671497642993927


In [41]:
Nombre = "Red_3"
Diseño = "C(32) + C(32) + Pool(2) + Flatten + Dense(128) + Dropout(0.5) + Dense(6)"
Estrategia_entrenamiento = "Entrenamiento desde 0, Dropout y EarlyStopping"

image_size = 32 #tamaño imagen
batch_size = 64 #tamaño del lote
kernel_size = 3 #tamaño kernel
pool_size = 2 #tamaño pool
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size)

convol3_dropout = Sequential()

convol3_dropout.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
convol3_dropout.add(Convolution2D (32, 
                              (kernel_size, kernel_size),
                              padding='valid',
                              input_shape=(image_size, image_size, 3), 
                              activation='relu'
                              ))

convol3_dropout.add(Convolution2D (32, (kernel_size, kernel_size), activation='relu'))
convol3_dropout.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
convol3_dropout.add(Flatten())
convol3_dropout.add(Dense(128, activation="relu"))
convol3_dropout.add(Dropout(0.5))
convol3_dropout.add(Dense(6, activation="softmax"))

convol3_dropout.summary()

convol3_dropout.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])


Es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10) #EarlyStopping para detener el entrenamiento
Mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True) #Guarda el mejor modelo

convol3_dropout.fit(train_dataset, validation_data = val_dataset, batch_size=64, epochs=50, verbose=2, callbacks=[Es, Mc])


convol3_dropout_Es = load_model('best_model.h5')

score_convol3_dropout = convol3_dropout_Es.evaluate(test_dataset)
print("Test loss", score_convol3_dropout[0])
print("Test accuracy", score_convol3_dropout[1]) #El accuray mejora, pero el modelo sigue sobreajustado

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_5 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 6272)              0         
                                                               

In [42]:
Nombre = "Red_4"
Diseño = "C(50) + Pool(3) + C(50) + Pool(2) + C(50) + Pool(2) + Flatten +  Dense(128) + Dense(256) + Dense(6)"
Estrategia_entrenamiento = "Entrenamiento desde 0 y EarlyStopping"

image_size = 128
batch_size = 128
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size) # Al entrenar con imágenes más grandes, sobreescribimos los datos train,val y test con imágenes más grandes.

kernel_size = 3
pool_size = 3
pool_size2 = 2
pool_size3 = 2

convol4 = Sequential()

convol4.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
convol4.add(Convolution2D (50, 
                          (kernel_size, kernel_size),
                          padding='valid',
                          input_shape=(image_size, image_size, 3), 
                          activation='relu'))
convol4.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
convol4.add(Convolution2D (50, (kernel_size, kernel_size), activation='relu'))
convol4.add(MaxPooling2D(pool_size=(pool_size2, pool_size2)))
convol4.add(Convolution2D (50, (kernel_size, kernel_size), activation='relu'))
convol4.add(MaxPooling2D(pool_size=(pool_size3, pool_size3)))
convol4.add(Flatten())
convol4.add(Dense(128, activation="relu"))
convol4.add(Dense(256, activation="relu"))
convol4.add(Dense(6, activation="softmax"))

convol4.summary()

convol4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

Es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
Mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
epochs = 50

convol4.fit(train_dataset, validation_data= val_dataset, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=[Es, Mc])

convol4_Es = load_model('best_model.h5')

score_convol4 = convol4_Es.evaluate(test_dataset)
print("Test loss", score_convol4[0])
print("Test accuracy", score_convol4[1]) #accuracy 0.54. El modelo mejora, pero sigue habiendo overfitting

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 126, 126, 50)      1400      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 42, 42, 50)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 40, 40, 50)        22550     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 20, 20, 50)       0         
 2D)                                                           

In [43]:
Nombre = "Red_5"
Diseño = "C(128) + Pool(3) + C(128) + Pool(3) + C(128) + Pool(3) + Flatten + Dense(256) + Dropout(0.5) + Dense(6)"
Estrategia_entrenamiento = "Entrenamiento desde 0, DropOut y EarlyStopping"

image_size = 256
batch_size = 128
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size)

kernel_size = 3
pool_size = 3


convol5 = Sequential()

convol5.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
convol5.add(Convolution2D (128, 
                          (kernel_size, kernel_size),
                          padding='valid',
                          input_shape=(image_size, image_size, 3), 
                          activation='relu'))
convol5.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
convol5.add(Convolution2D (128, (kernel_size, kernel_size), activation='relu'))
convol5.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
convol5.add(Convolution2D (128, (kernel_size, kernel_size), activation='relu'))
convol5.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
convol5.add(Flatten())
convol5.add(Dense(256, activation="relu"))
convol5.add(Dropout(0.5))
convol5.add(Dense(6, activation="softmax"))

convol5.summary()

convol5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

Es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
Mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
epochs = 50

convol5.fit(train_dataset, validation_data= val_dataset, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=[Es, Mc])

convol5_Es = load_model('best_model.h5')

score_convol5 = convol5_Es.evaluate(test_dataset)
print("Test loss", score_convol5[0])
print("Test accuracy", score_convol5[1])

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_7 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_11 (Conv2D)          (None, 254, 254, 128)     3584      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 84, 84, 128)      0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 82, 82, 128)       147584    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 27, 27, 128)      0         
 2D)                                                           

## Transfer learning

Aunque diseñar nuestra propia red puede producir algunos resultados aceptables, suele ser mejor aprovechar el conocimiento ya existente en una red pre-entrenada. Esto no solo nos lleva a resultados mejores, sino que además nos ahorra mucho tiempo de diseño de la red. Para ello, el módulo [Keras Applications](https://keras.io/api/applications/) contiene varios diseños de redes listos para su uso. Por ejemplo, para hacer uso de la famosa red VGG16 hacemos lo siguiente

In [44]:
from tensorflow.keras.applications import VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

Por defecto, todas las redes de Keras Applications están precargadas con los pesos que se obtuvieron al entrenar la red sobre el dataset de la competición de [ImageNet](http://www.image-net.org/). Para adaptar la red a nuestro problema, hemos necesitado especificar la resolución de nuestras imágenes (`input_shape`), así como eliminar las capas de salida (`top`) de la red original, dado que nosotros tendremos un número diferente de clases.

Ahora, ¿cómo hacemos para transferir el aprendizaje de esta red? Vamos a ver cómo implementar la estrategia de "bottleneck features". En primer lugar, marcamos el modelo VGG16 como no entrenable, para que sus parámetros se mantengan congelados.

In [45]:
vgg16_model.trainable = False

Hecho esto, vamos a construir una red neuronal que incluya la VGG16 como una de sus "capas". Es necesario tener en cuenta que la red VGG16 se entrenó realizando una normalización muy específica de las imágenes de entrenamiento, y nosotros debemos seguir ese mismo proceso para que la red se comporte correctamente. Convenientemente, Keras también nos da una funcionalidad para replicar la normalización que la VGG16 necesita.

In [46]:
from tensorflow.keras.applications.vgg16 import preprocess_input

Podemos probar esta normalización con alguna de las imágenes de nuestro dataset

In [47]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

Before normalizing: [[[170.   188.   188.  ]
  [171.   189.   189.  ]
  [171.   189.   189.  ]]

 [[170.   188.   188.  ]
  [171.   189.   189.  ]
  [171.75 189.75 189.75]]

 [[170.   188.   188.  ]
  [171.5  189.5  189.5 ]
  [172.   190.   190.  ]]]
After normalizing: [[[84.061 71.221 46.32 ]
  [85.061 72.221 47.32 ]
  [85.061 72.221 47.32 ]]

 [[84.061 71.221 46.32 ]
  [85.061 72.221 47.32 ]
  [85.811 72.971 48.07 ]]

 [[84.061 71.221 46.32 ]
  [85.561 72.721 47.82 ]
  [86.061 73.221 48.32 ]]]


La normalización realizada por la VGG16 consiste en invertir el orden de los canales de color (RGB -> BGR), y restar los valores medios sobre todo el dataset ImageNet para cada canal de color por separado. Afortunadamente, la función `preprocess_input` que hemos importado hace todo este trabajo por nosotros. Además, podemos incrustar esta función como la primera capa de nuestra red, cumpliendo el papel de la capa `Rescaling` que utilizamos en el apartado anterior. Esto es posible gracias a la capa `Lambda`, que permite construir una capa Keras en base a cualquier función de Tensorflow. De modo que, vamos a comenzar nuestro diseño de red con esta capa.

In [48]:
from tensorflow.keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Tras esto, podemos añadir toda la red VGG16 como si fuera una nueva capa, y nuestras propias capas después de ella. A continuación tenemos un ejemplo de esta clase de diseño, aunque es importante destacar que es un diseño muy sencillo que contiene algunos fallos; una red real para hacer transfer learning debería tener un diseño mejor.

In [49]:
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 256, 256, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_9 (Flatten)         (None, 32768)             0         
                                                                 
 dense_14 (Dense)            (None, 6)                 196614    
                                                                 
Total params: 14,911,302
Trainable params: 196,614
Non-trainable params: 14,714,688
_________________________________________________________________


Observa cómo en el resumen del modelo podemos ver que la red completa tiene millones de parámetros, pero dado que hemos congelado toda la parte de la red perteneciente a la VGG16, solo unos pocos miles de parámetros son entrenables (trainable): aquellos correspondientes a la capa Dense que hemos colocado al final.

Ya podemos compilar y entrenar el modelo a la manera habitual.

In [50]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

17/17 [==============================] - 19s 978ms/step - loss: 79.4171 - accuracy: 0.7599
Loss 79.4, accuracy 76.0%


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Usando la estrategia "bottleneck" presentada, implementa una red que haga transfer learning desde la red VGG16, con un diseño correcto. Si lo haces adecuadamente, esta red debe obtener mejores resultados que con la red que diseñaste en el apartado anterior, y con al menos un 80% de acierto sobre el conjunto de test.
</font>

***

In [65]:
Nombre = "VGG16_1"
Diseño = "VGG16  + Flatten + Dense(128) + DropOut(0.5) + Dense(128) + DropOut(0.5) + Dense(128) + DropOut(0.5) + Dense(6)"
Estrategia_entrenamiento = "Bottleneck, Aumento, DropOut y EarlyStopping"

image_size = 128
batch_size = 128
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size)

channels = 3
kernel_size = 3 


VGG16_1 = Sequential()
VGG16_1.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

VGG16_1.add(vgg16_model)
VGG16_1.add(Flatten())
VGG16_1.add(Dense(128, activation="relu"))
VGG16_1.add(Dropout(0.5))
VGG16_1.add(Dense(128, activation="relu"))
VGG16_1.add(Dropout(0.5))
VGG16_1.add(Dense(128, activation="relu"))
VGG16_1.add(Dropout(0.5))
VGG16_1.add(Dense(6, activation= 'softmax'))

VGG16_1.summary()

VGG16_1.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=["accuracy"])

Es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
Mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
epochs = 50

VGG16_1.fit(train_dataset, validation_data=(val_dataset), batch_size=batch_size, steps_per_epoch=6082/batch_size, epochs = epochs, verbose=2, callbacks=[Es, Mc])

VGG16_1_Es = load_model('best_model.h5')

score_VGG16_1 = VGG16_1_Es.evaluate(test_dataset)
print("Test loss", score_VGG16_1[0])
print("Test accuracy", score_VGG16_1[1])




Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_12 (Lambda)          (None, 128, 128, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_20 (Flatten)        (None, 8192)              0         
                                                                 
 dense_51 (Dense)            (None, 128)               1048704   
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_52 (Dense)            (None, 128)               16512  


Epoch 2: val_accuracy did not improve from 0.23624
47/47 - 5s - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 1.7684 - val_accuracy: 0.2362 - 5s/epoch - 104ms/step
17/17 [==============================] - 6s 219ms/step - loss: 1.7756 - accuracy: 0.2401
Test loss 1.7755861282348633
Test accuracy 0.24009661376476288


In [66]:
from tensorflow.keras.layers import GlobalAveragePooling2D

Nombre = "VGG16_2"
Diseño = "VGG16  + C(128) + Pool(3) + C(128) + Pool(3) + GAP2D + Dense(512) + DropOut(0.5) + Dense(512) + DropOut(0.5) + Dense(512) + DropOut(0.5) + Dense(6)"
Estrategia_entrenamiento = "Bottleneck, Aumento, DropOut, EarlyStopping y GlobalAveragePooling2D"

image_size = 256
batch_size = 128
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size, batch_size)

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

vgg16_model.trainable = False
from keras.applications.vgg16 import preprocess_input



channels = 3
pool_size = 3 
kernel_size = 3 


VGG16_2 = Sequential()

VGG16_2.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

VGG16_2.add(vgg16_model)

VGG16_2.add(Convolution2D(128,
                           (kernel_size, kernel_size),
                           padding='same',
                           activation='relu'))
VGG16_2.add(MaxPooling2D(pool_size=pool_size, strides=4))
VGG16_2.add(GlobalAveragePooling2D(data_format=None, keepdims=False))

VGG16_2.add(Dense(512, activation='relu'))
VGG16_2.add(Dropout(0.5))

VGG16_2.add(Dense(512, activation='relu'))
VGG16_2.add(Dropout(0.5))

VGG16_2.add(Dense(512, activation='relu'))
VGG16_2.add(Dropout(0.5))

VGG16_2.add(Dense(6, activation="softmax"))


VGG16_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

Es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
Mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
epochs = 50

VGG16_2.fit(train_dataset, validation_data=val_dataset, epochs=epochs, callbacks=[Es, Mc])

VGG16_2_Es = load_model('best_model.h5')

score_VGG16_2 = VGG16_2_Es.evaluate(test_dataset)
print("Test loss", score_VGG16_2[0])
print("Test accuracy", score_VGG16_2[1])

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Epoch 1/50
48/48 [==============================] - ETA: 0s - loss: 2.7278 - accuracy: 0.3357
Epoch 1: val_accuracy improved from -inf to 0.59535, saving model to best_model.h5
48/48 [==============================] - 70s 1s/step - loss: 2.7278 - accuracy: 0.3357 - val_loss: 1.1278 - val_accuracy: 0.5954
Epoch 2/50
48/48 [==============================] - ETA: 0s - loss: 0.9986 - accuracy: 0.6493
Epoch 2: val_accuracy improved from 0.59535 to 0.74288, saving model to best_model.h5
48/48 [==============================] - 57s 1s/step - loss: 0.9986 - accuracy: 0.6493 - val_loss: 0.7328 - val_accuracy: 0.7429
Epoch 3/50
48/48 [==============================] - ETA: 0s - loss: 0.6872 - accuracy: 0.7738
Epoch 3: val_accuracy improved from 0.74288 to 0.78083, saving model to best_model.h5
48/48 [==============================] - 57s 1s/step - loss: 0.6872 - accuracy: 0.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunos consejos para mejorar tu diseño de red:
    
- Incluye una o más capas Dense, con sus funciones de activación apropiadas, antes de la capa de salida.
- Intenta usar una capa de tipo [GlobalAveragePooling](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) en lugar de la capa Flatten. Esta capa calcula una media de todos los valores de píxeles para cada canal, y en algunas ocasiones produce mejores resultados que la capa Flatten.
- ¡Y no olvides todos los consejos del apartado anterior! También aplican aquí.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>

Para mejorar aún más los resultados de tu red, utiliza las siguientes ideas:

- Usa las estrategias PRO del ejercicio anterio.
- Prueba otras redes pre-entrenadas de <a href="https://keras.io/api/applications/">Keras Applications</a>, como ResNet, Xception o EfficientNet.
- Usa una estrategia de transfer learning más avanzada, como fine-tuning o una combinación de bottleneck features y fine-tuning. Revisa las diapositivas de clase para saber cómo.
   
Si empleas todos estos trucos, es posible alcanzar más de un 90% de acierto en el conjunto de test.

</font>

***

In [6]:
import pandas as pd

model_names = ["Red Sencilla", "Red Sencilla(10)", "Red 1", "Red2", "Red3","Red 4","Red5","VGG16_1","VGG16_2"]
test_loss = [1.50, 1.58, 1.50,1.99,1.41,1.28,1.17,1.77,0.59]
test_accuracy = [0.39, 0.39, 0.39, 0.46, 0.45,0.54,0.55,0.24,0.82]
Estrategia = ["Entrenamiento desde 0", "Entrenamiento desde 0", "Entrenamiento desde 0", "Entrenamiento desde 0 y Dropout","Entrenamiento desde 0, Dropout y EarlyStopping","Entrenamiento desde 0, Dropout y EarlyStopping","Entrenamiento desde 0, Dropout y EarlyStopping","Bottleneck, Aumento, DropOut y EarlyStopping","Bottleneck, Aumento, DropOut, EarlyStopping y GlobalAveragePooling2D"]
Diseño = ["C(4) + Flatten + Dense(6)","C(4) + Flatten + Dense(6)","C(32) + Pool(2) + Flatten + Dense(6)","C(32) + C(32) + Pool(2) + Flatten + Dense(128) + Dropout(0.5) + Dense(6)","C(32) + C(32) + Pool(2) + Flatten + Dense(128) + Dropout(0.5) + Dense(6)","C(50) + Pool(3) + C(50) + Pool(2) + C(50) + Pool(2) + Flatten +  Dense(128) + Dense(256) + Dense(6)","C(128) + Pool(3) + C(128) + Pool(3) + C(128) + Pool(3) + Flatten + Dense(256) + Dropout(0.5) + Dense(6)","VGG16  + Flatten + Dense(128) + DropOut(0.5) + Dense(128) + DropOut(0.5) + Dense(128) + DropOut(0.5) + Dense(6)","VGG16  + C(128) + Pool(3) + C(128) + Pool(3) + GAP2D + Dense(512) + DropOut(0.5) + Dense(512) + DropOut(0.5) + Dense(512) + DropOut(0.5) + Dense(6)"]



df_results = pd.DataFrame({
    'Model Name': model_names,
    'Test Loss': test_loss,
    'Test Accuracy': test_accuracy,
    'Estrategia': Estrategia,
    'Diseño': Diseño
})
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df_results


,Model Name,Test Loss,Test Accuracy,Estrategia,Diseño
0,Red Sencilla,1.50,0.39,Entrenamiento desde 0,C(4) + Flatten + Dense(6)
1,Red Sencilla(10),1.58,0.39,Entrenamiento desde 0,C(4) + Flatten + Dense(6)
2,Red 1,1.50,0.39,Entrenamiento desde 0,C(32) + Pool(2) + Flatten + Dense(6)
3,Red2,1.99,0.46,Entrenamiento desde 0 y Dropout,C(32) + C(32) + Pool(2) + Flatten + Dense(128) + Dropout(0.5) + Dense(6)
4,Red3,1.41,0.45,"Entrenamiento desde 0, Dropout y EarlyStopping",C(32) + C(32) + Pool(2) + Flatten + Dense(128) + Dropout(0.5) + Dense(6)
5,Red 4,1.28,0.54,"Entrenamiento desde 0, Dropout y EarlyStopping",C(50) + Pool(3) + C(50) + Pool(2) + C(50) + Pool(2) + Flatten + Dense(128) + Dense(256) + Dense(6)
6,Red5,1.17,0.55,"Entrenamiento desde 0, Dropout y EarlyStopping",C(128) + Pool(3) + C(128) + Pool(3) + C(128) + Pool(3) + Flatten + Dense(256) + Dropout(0.5) + Dense(6)
7,VGG16_1,1.77,0.24,"Bottleneck, Aumento, DropOut y EarlyStopping",VGG16 + Flatten + Dense(128) + DropOut(0.5) + Dense(128) + DropOut(0.5) + Dense(128) + DropOut(0.5) + Dense(6)
8,VGG16_2,0.59,0.82,"Bottleneck, Aumento, DropOut, EarlyStopping y GlobalAveragePooling2D",VGG16 + C(128) + Pool(3) + C(128) + Pool(3) + GAP2D + Dense(512) + DropOut(0.5) + Dense(512) + DropOut(0.5) + Dense(512) + DropOut(0.5) + Dense(6)


## Informe final y resumen de resultados

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Escribe en la siguiente celda un pequeño informe con:
    <ul>
        <li>Una tabla de resultados, indicando qué diseños de red has probado y qué resultados en test has obtenido. Puede usar un estilo de tabla como el que se muestra abajo.</li>
        <li>De las estrategias y diseños que has ido probando, ¿qué ha funcionado y qué no?</li>
        <li>¿Qué has aprendido con esta práctica?
    </ul>
</font>

***

In [ ]:
#la función de activación Relu produce sobreajuste(muy buen acierto en train y muy malo en test) y debemos introducir dropout para balancear el ajuste
#Cada vez que ejecuto un modelo no me salen valores iguales a pesar del random.set(seed).
#Transfer learning mejora mucho los resultados(sobretodo con GlobalAveragePooling )
#El earlystopping me parece un muy buen método. Detiene el sobreajuste, ahorra tiempo y recursos y produce una mayor estabilidad en el entrenamiento.
#he aprendido que el earlystopping es ventajoso para no perder tiempo.
#Se podría mejorar mucho más el accuracy con redes más complejas 

Ejemplo para la tabla de resultados

|Procesado de imágenes|Diseño de red neuronal|Estrategia de entrenamiento|Acierto en test|
|---------------------|----------------------|---------------------------|---------------|
|Tamaño 32x32, batch size 16|Convolutional(32) + Flatten + Dense(64)|Entrenamiento desde 0|xx%|
|Tamaño 64x64, batch size 32|VGG16 + Flatten + Dense(32)|Bottleneck features|yy%|
|...|...|...|...|